In [ ]:
import time
start = time.time()

import os
import pandas as pd
import numpy as np

from math import nan
from future.utils import iteritems

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style = "darkgrid")

%tensorflow_version 1.x

!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install sklearn_crfsuite
!pip install seqeval

import keras as k
from keras.models import Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.embeddings import Embedding

from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import train_test_split



from keras.layers import merge, Activation, Convolution1D, Dense, Dropout, Lambda, Convolution2D, Convolution3D, concatenate, Concatenate, GlobalMaxPooling1D
from keras.models import Sequential



import warnings
warnings.filterwarnings("ignore")

TensorFlow 1.x selected.
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-wx_5o7x5
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-wx_5o7x5
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=4cbcc7d7e76f8fbd6b09f807e0d2c470bebaaceda76fd33d763addbd8a5504eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-ltbn5nr9/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
cd gdrive/MyDrive/coursework

/content/gdrive/MyDrive/coursework


In [ ]:
from scienceie_loader import load_tokenized_data, load_data_with_char_offsets, get_entity_span_from_B_index

In [ ]:
data_root = os.path.join(os.getcwd(), 'original_datasets')
data_train = os.path.join(data_root, 'scienceie2017_train/train2')
data_dev = os.path.join(data_root, 'scienceie2017_dev/dev')
data_test = os.path.join(data_root, 'semeval_articles_test')

In [ ]:
train_docs, train_rels, _ = load_tokenized_data(data_train)
dev_docs, dev_rels, _ = load_tokenized_data(data_dev)
test_docs, test_rels, _ = load_tokenized_data(data_test)

In [ ]:
train = [j for i in train_docs for j in i]
test = [j for i in test_docs for j in i]

In [ ]:
len(train)

62475

In [ ]:
len(test)

21133

In [ ]:
dataset_train = pd.DataFrame(train, columns = ["Words", "Tags"])
dataset_test = pd.DataFrame(test, columns = ["Words", "Tags"])

In [ ]:
ne_tags_train = {tag for word, tag in train}
ne_tags_test = {tag for word, tag in train}

In [ ]:
ne_tags_train = list(ne_tags_train)
ne_tags_test = list(ne_tags_test)

In [ ]:
dataset_train.info()
dataset_train.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62475 entries, 0 to 62474
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Words   62475 non-null  object
 1   Tags    62475 non-null  object
dtypes: object(2)
memory usage: 976.3+ KB


,Words,Tags
45144,a,I-Material
29948,the,O
3693,range,O
41244,notion,B-Task
14061,in,O


In [ ]:
r = []
for i in range(len(train_docs)):
    x = len(train_docs[i])
    for j in range(x):
        r.append(i+1)

In [ ]:
dataset_train["Sentence_idx"] = r

In [ ]:
class make_Sentence(object):

  def __init__(self, dataset):
    self.sent = 1
    self.dataset = dataset
    self.empty = False
    func = lambda s : [(word, tag) for word, tag in zip(s["Words"].values.tolist(),
                                                        s["Tags"].values.tolist())]
    self.group = self.dataset.groupby("Sentence_idx").apply(func)
    self.sentences = [s for s in self.group]

  def get_next(self):
    try:
      s = self.group["Sentence : {}".format(self.sent)]
      self.sent += 1
      return s
    except :
      return None

In [ ]:
getter = make_Sentence(dataset_train)
sentences_train = getter.sentences

In [ ]:
words_train = list(set(dataset_train["Words"].values))
n_words_train = len(words_train)

tags_train = []
for tag_train in set(dataset_train["Tags"].values):
  if tag_train is nan or isinstance(tag_train, float):
    tags_train.append("Unknown")
  else:
    tags_train.append(tag_train)

n_tags_train  = len(tags_train)

In [ ]:
word2idx_train = {w: i for i, w in enumerate(words_train)}
tag2idx_train = {t: i for i, t in enumerate(tags_train)}
idx2tag_train = {v: k for k, v in iteritems(tag2idx_train)}

In [ ]:
maxlen_train = 344

X_train = [[word2idx_train[w[0]] for w in s] for s in sentences_train]
X_train = pad_sequences(maxlen = maxlen_train, sequences = X_train, padding = "post",value = n_words_train - 1)

Y_train = [[tag2idx_train[w[1]] for w in s] for s in sentences_train]
Y_train = pad_sequences(maxlen = maxlen_train, sequences = Y_train, padding = "post", value = 6)
Y_train = [to_categorical(i, num_classes = n_tags_train) for i in Y_train]

In [ ]:
input = Input(shape=(344,))
word_embedding_size = 210

In [ ]:
loss_function = 'categorical_crosswntropy'

In [ ]:
def build_cnn_char(input_dim, output_dim,nb_filter):
    clf = Sequential()
    clf.add(Embedding(input_dim=n_words_train, output_dim=word_embedding_size, input_length=344))
    clf.add(Convolution1D(filters = 16,
                          filter_length = 4,border_mode="same",activation="relu",subsample_length=1))
    #clf.add(GlobalMaxPooling1D())
    clf.add(Dense(100))
    clf.add(Dropout(0.2))
    clf.add(Activation("tanh"))
    clf.add(Dense(output_dim=output_dim, activation='softmax'))

    clf.compile(optimizer='adagrad',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    return clf


In [ ]:
cnn = build_cnn_char(X_train, 7, 210)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
def max_1d(X):
    return K.max(X,axis=1)

In [ ]:
maxlen = 344

def build_cnn_char_complex(input_dim, output_dim, nb_filter):
    randomEmbeddingLayer = Embedding(input_dim = n_words_train, output_dim = word_embedding_size, input_length = 344)
    poolingLayer = Lambda(max_1d, output_shape = (210,))
    conv_filters = []
    for n_gram in range(2,4):
        ngramModel = Sequential()
        ngramModel.add(randomEmbeddingLayer)
        ngramModel.add(Convolution1D(nb_filter = nb_filter,
                                     filter_length = n_gram,
                                     border_mode = "valid",
                                     activation = "relu",
                                     subsample_length = 1))
        ngramModel.add(poolingLayer)
        conv_filters.append(ngramModel)
    
    clf = Sequential()
    clf.add(Merge(conv_filters,mode="concat"))
    clf.add(Activation("relu"))
    clf.add(Dense(100))
    clf.add(Dropout(0.1))
    clf.add(Activation("tanh"))
    clf.add(Dense(output_dim = output_dim, activation = 'softmax'))

    clf.compile(optimizer='adagrad',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    return clf

In [ ]:
cnn = build_cnn_char(X_train, 7, 210)
#cnn = build_cnn_char_complex(X_train, 7, 210)

In [ ]:
cnn.fit(np.array(X_train),np.array(Y_train),26, epochs = 20)    #'16 --> 26'

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
350/350 [==============================] - 2s 6ms/step - loss: 0.8152 - accuracy: 0.7783
Epoch 2/20
350/350 [==============================] - 1s 4ms/step - loss: 0.4547 - accuracy: 0.8468
Epoch 3/20
350/350 [==============================] - 1s 4ms/step - loss: 0.4102 - accuracy: 0.8570
Epoch 4/20
350/350 [==============================] - 1s 4ms/step - loss: 0.3772 - accuracy: 0.8655
Epoch 5/20
350/350 [==============================] - 1s 4ms/step - loss: 0.3469 - accuracy: 0.8750
Epoch 6/20
350/350 [==============================] - 1s 4ms/step - loss: 0.3178 - accuracy: 0.8849
Epoch 7/20
350/350 [==============================] - 1s 4ms/step - loss: 0.2885 - accuracy: 0.8975
Epoch 8/20
350/350 [==============================] - 1s 4ms/step - loss: 0.2618 - accuracy: 0.9091
Epoch 9/20
350/350 [==============================] - 1s 4ms/step - loss: 0.2370 - accuracy: 0.9206
Epoch

In [ ]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 344, 210)          2098740   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 344, 16)           13456     
_________________________________________________________________
dense_3 (Dense)              (None, 344, 100)          1700      
_________________________________________________________________
dropout_2 (Dropout)          (None, 344, 100)          0         
_________________________________________________________________
activation_2 (Activation)    (None, 344, 100)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 344, 7)            707       
Total params: 2,114,603
Trainable params: 2,114,603
Non-trainable params: 0
____________________________________________

In [ ]:
cnn.fit(np.array(X_train),np.array(Y_train),26, epochs = 10)    #'16 --> 26'

Epoch 1/10
350/350 [==============================] - 1s 4ms/step - loss: 0.1025 - accuracy: 0.9694
Epoch 2/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0980 - accuracy: 0.9709
Epoch 3/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0942 - accuracy: 0.9715
Epoch 4/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0903 - accuracy: 0.9729
Epoch 5/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0875 - accuracy: 0.9738
Epoch 6/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0840 - accuracy: 0.9745
Epoch 7/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0817 - accuracy: 0.9755
Epoch 8/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0789 - accuracy: 0.9759
Epoch 9/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0768 - accuracy: 0.9769
Epoch 10/10
350/350 [==============================] - 1s 4ms/step - loss: 0.0747 - accuracy: 0.9771

In [ ]:
X_train.shape

(350, 344)

In [ ]:
dataset_test.info()
dataset_test.sample(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21133 entries, 0 to 21132
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Words   21133 non-null  object
 1   Tags    21133 non-null  object
dtypes: object(2)
memory usage: 330.3+ KB


,Words,Tags
3397,cuprite,B-Material
20629,it,O
7155,previous,O
11298,resp.,O
258,number,O
9842,reactions.,O
3950,through,O


In [ ]:
r = []
for i in range(len(test_docs)):
    x = len(test_docs[i])
    for j in range(x):
        r.append(i+1)

In [ ]:
dataset_test["Sentence_idx"] = r

In [ ]:
dataset_test.info()
dataset_test.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21133 entries, 0 to 21132
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Words         21133 non-null  object
 1   Tags          21133 non-null  object
 2   Sentence_idx  21133 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 495.4+ KB


,Words,Tags,Sentence_idx
19932,",",O,94
806,overlay,O,4
7534,so,O,34
20747,will,O,98
19828,control,O,93


In [ ]:
getter = make_Sentence(dataset_test)
sentences_test = getter.sentences

In [ ]:
words_test = list(set(dataset_test["Words"].values))
n_words_test = len(words_test)

tags_test = []
for tag_test in set(dataset_test["Tags"].values):
  if tag_test is nan or isinstance(tag_test, float):
    tags_test.append("Unknown")
  else:
    tags_test.append(tag_test)

n_tags_test  = len(tags_test)

In [ ]:
word2idx_test = {w: i for i, w in enumerate(words_test)}
tag2idx_test = {t: i for i, t in enumerate(tags_test)}
idx2tag_test = {v: k for k, v in iteritems(tag2idx_test)}

In [ ]:
maxlen_test = 344

X_test = [[word2idx_test[w[0]] for w in s] for s in sentences_test]
X_test = pad_sequences(maxlen = maxlen_test, sequences = X_test, padding = "post",value = n_words_test - 1)

Y_test = [[tag2idx_test[w[1]] for w in s] for s in sentences_test]
Y_test = pad_sequences(maxlen = maxlen_test, sequences = Y_test, padding = "post", value = 6)
Y_test = [to_categorical(i, num_classes = n_tags_test) for i in Y_test]

In [ ]:
x = cnn.predict(X_test)

In [ ]:
def prediction(pred):
  output = []
  for i in pred:
    output_i = []
    for p in i:
      i = np.argmax(p)
      output_i.append(idx2tag_test[i])
    output.append(output_i)
  return output

pred_test = cnn.predict(X_test, verbose = 1)
labels_pred = prediction(pred_test)
labels_test = prediction(Y_test)

100/100 [==============================] - 0s 672us/step


In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.1%}".format(f1_score(labels_test, labels_pred)))

F1-score: 0.7%
